### svm

利用SMO方法实现svm

In [ ]:
import numpy as np
import time

---
#### 一些基本的辅助函数

In [ ]:
def loadDataset(filename):
    dataMat=[]
    labelMat=[]
    fr=open(filename)
    for line in fr.readlines():
        print(line)
        lineArr=line.strip().split('\t')
        dataMat.append([float(lineArr[0]),float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat,labelMat

X_train,y_train=loadDataset('testSet.txt')

In [ ]:
# 返回在0~m样本范围内,和i不同的随机的一个值
def selectJrand(i,m):
    j=i
    while(j==i):
        j=int(np.random.uniform(0,m))
    
    return j


In [ ]:
# 把alpha clip到H和L范围内
def clipAlpha(aj,H,L):
    if aj>H:
        aj=H
    elif aj<L:
        aj=L
    return aj


In [ ]:
# dataSe: 是数据集
# labels: 是标签,+1和-1
# C是 alpha的最大值,约束条件
# toler 误差容忍
# maxIter: alpha不再变化的最大迭代次数,尝试这么多次的迭代alpha都不在变化后,认为找到最优的alpha.
def smo(dataSet,labels,C,toler,maxIter):
    X=np.mat(dataSet)
    y=np.mat(labels).transpose()
    b=0
    m,n=X.shape
    #alphas=np.zeros([m,1],np.float32)
    alphas = np.mat(np.zeros((m,1)))
    iterI=0
    print(maxIter)
    while(iterI<maxIter):# end
        # update
        alphaPairsChanged=0
        for i in range(m):# end
            fXi=float(np.multiply(alphas,y).T*(X*X[i,:].T))+b
            
            Ei=fXi-float(y[i])
            # KKT conditions
            # 第一: 偏差大于 toler. y[i]表符号.
            # 第二: alphas[i]满足0~C,而且不能取边界; 这个边界问题我还没搞懂
            if ((y[i]*Ei<-toler) and alphas[i]<C ) or ((y[i]*Ei>toler)and alphas[i]>0): #kkt条件,参考<<支持向量机通俗导论>>及<<支持向量机导论>>
                j=selectJrand(i,m)
                fXj=float(np.multiply(alphas,y).T*(X*X[j,:].T))+b
                Ej=fXj-y[j]
                alpha_i_old=alphas[i].copy()
                alpha_j_old=alphas[j].copy()
                if y[i]!=y[j]:
                    L=max(0,alphas[j]-alphas[i])
                    H=min(C,C+alphas[j]-alphas[i])
                else:
                    L=max(0,alphas[i]+alphas[j]-C)
                    H=min(C,alphas[j]-alphas[i])
                
                eta=2.0*X[i,:]*X[j,:].T - X[i,:]*X[i,:].T - X[j,:]*X[j,:].T
                #eta = 2.0 * dataMatrix[i,:]*dataMatrix[j,:].T - dataMatrix[i,:]*dataMatrix[i,:].T - dataMatrix[j,:]*dataMatrix[j,:].T
                if eta>=0:
                    print("eta >=0")
                    continue
                alphas[j]-=y[j]*(Ei-Ej)/eta # 更新alpha[j]
                alphas[j]=clipAlpha(alphas[j],H,L)
                if(np.abs(alphas[j]-alpha_j_old)<0.00001):
                    print("j not move")
                    continue
                alphas[i]+=y[j]*y[i]*(alpha_j_old-alphas[j])
                
                b1=b-Ei-y[i]*(alphas[i]-alpha_i_old)*X[i,:]*X[i,:].T-y[j]*(alphas[j]-alpha_j_old)*X[j,:]*X[j,:].T
                b2=b-Ej-y[i]*(alphas[i]-alpha_i_old)*X[i,:]*X[i,:].T-y[j]*(alphas[j]-alpha_j_old)*X[j,:]*X[j,:].T
                if (0<alphas[i]) and (C>alphas[i]):
                    b=b1
                elif (0<alphas[j]) and (C>alphas[j]):
                    b=b2
                else:
                    b=(b1+b2)/2.0
                    
                alphaPairsChanged+=1
                print("iter: %d i:%d, pairs changed %d" % (iterI,i,alphaPairsChanged))
        if alphaPairsChanged==0:
            iterI+=1
        else:
            iterI=0
        print("iteration num: %d"%iterI)
    
    return b,alphas
                
                
            
        # break
    
b,alphas=smo(X_train,y_train,0.6,0.001,40)    